In [1]:
import pandas as pd

In [25]:
df = pd.read_csv("data/cell_data.csv", sep=';')

df.drop(["Label", "Object ID", "Parent Trace ID"], axis = 1, inplace = True)
df

,Total Intensity CSU 561,Mean Intensity CSU 561,Total Intensity CSU 488,Mean Intensity CSU 488,Total Intensity CSU 405,Mean Intensity CSU 405,Area,Parent Object ID (MO),cell cycle
0,1218438.0,84.320.972,2302999,15.937.709,3729942.0,25.812.747,1445,47,G2
1,618824.0,8.654.881,1074270,15.024.755,1708607.0,23.896.602,715,47,G2
2,26242518.0,15.620.547,830844,49.454.999,3553084.0,21.149.309,1680,59,G1
3,1375343.0,51.472.418,3123572,11.690.015,7169822.0,26.833.167,2672,59,G2
4,2498555.0,16.266.635,229999,14.973.894,3653036.0,23.782.786,1536,69,G1
...,...,...,...,...,...,...,...,...,...
1139,2407608.0,14.170.736,206436,12.150.442,4033081.0,23.737.969,1699,2927,G1
1140,1773447.0,98.964.679,3604093,20.112.126,6446149.0,35.971.814,1792,2930,G2
1141,1529220.0,10.060.658,3613849,23.775.322,6207942.0,40.841.724,1520,2930,G2
1142,3609520.0,18.557.943,2135528,10.979.579,4840860.0,2.488.874,1945,2933,S


In [45]:
groupby_count = df.groupby(["Parent Object ID (MO)"])["cell cycle"].count().reset_index()

In [49]:
not_2_nuclear = list(groupby_count.loc[groupby_count["cell cycle"] != 2, "Parent Object ID (MO)"])

In [51]:
df_2_nuclear = df[~df["Parent Object ID (MO)"].isin(not_2_nuclear)]

In [53]:
print("%d objects not conform out of %d total" %(df.shape[0] - df_2_nuclear.shape[0], df.shape[0]))

238 objects not conform out of 1144 total


In [69]:
similar_cell_cycle = df_2_nuclear[df_2_nuclear.duplicated(["Parent Object ID (MO)", "cell cycle"], keep = False)]
not_similar_cell_cycle = df_2_nuclear[~df_2_nuclear.duplicated(["Parent Object ID (MO)", "cell cycle"], keep = False)]

print("Cells with similar nuclear cell cycle: %.1f%%" %(similar_cell_cycle.shape[0]/df_2_nuclear.shape[0]*100)) 
print("Cells with not similar nuclear cell cycle: %.1f%%" %(not_similar_cell_cycle.shape[0]/df_2_nuclear.shape[0]*100)) 

Cells with similar nuclear cell cycle: 89.6%
Cells with not similar nuclear cell cycle: 10.4%


In [71]:
df_G1_G1 = similar_cell_cycle[similar_cell_cycle["cell cycle"] == "G1"]
df_G2_G2 = similar_cell_cycle[similar_cell_cycle["cell cycle"] == "G2"]
df_S_S = similar_cell_cycle[similar_cell_cycle["cell cycle"] == "S"]

In [73]:
print("Cells with double G1/G1 nuclear cell cycle: %.1f%% (%.1f%%)" %(df_G1_G1.shape[0]/df_2_nuclear.shape[0]*100, df_G1_G1.shape[0]/similar_cell_cycle.shape[0]*100)) 
print("Cells with double G2/G2 nuclear cell cycle: %.1f%% (%.1f%%)" %(df_G2_G2.shape[0]/df_2_nuclear.shape[0]*100, df_G2_G2.shape[0]/similar_cell_cycle.shape[0]*100)) 
print("Cells with double S/S nuclear cell cycle: %.1f%% (%.1f%%)" %(df_S_S.shape[0]/df_2_nuclear.shape[0]*100, df_S_S.shape[0]/similar_cell_cycle.shape[0]*100)) 

Cells with double G1/G1 nuclear cell cycle: 51.9% (57.9%)
Cells with double G2/G2 nuclear cell cycle: 11.7% (13.1%)
Cells with double S/S nuclear cell cycle: 26.0% (29.1%)


In [91]:
lists_dict = {"G1_G2": [], "G1_S": [], "G2_S": []}

for id in not_similar_cell_cycle["Parent Object ID (MO)"].unique():
    mask = not_similar_cell_cycle[not_similar_cell_cycle["Parent Object ID (MO)"] == id] 
    list_c_cycle = list(mask["cell cycle"])
    
    if ("G1" in list_c_cycle and "G2" in list_c_cycle) or ("G2" in list_c_cycle and "G1" in list_c_cycle):
        lists_dict["G1_G2"].append(id)
    elif ("G1" in list_c_cycle and "S" in list_c_cycle) or ("S" in list_c_cycle and "G1" in list_c_cycle):
        lists_dict["G1_S"].append(id)
    elif ("G2" in list_c_cycle and "S" in list_c_cycle) or ("S" in list_c_cycle and "G2" in list_c_cycle):
        lists_dict["G2_S"].append(id)

df_G1_G2 = not_similar_cell_cycle[not_similar_cell_cycle["Parent Object ID (MO)"].isin(lists_dict["G1_G2"])]
df_G1_S = not_similar_cell_cycle[not_similar_cell_cycle["Parent Object ID (MO)"].isin(lists_dict["G1_S"])]
df_G2_S = not_similar_cell_cycle[not_similar_cell_cycle["Parent Object ID (MO)"].isin(lists_dict["G2_S"])]

In [92]:
print("Cells with G1/G2 nuclear cell cycle: %.1f%% (%.1f%%)" %(df_G1_G2.shape[0]/df_2_nuclear.shape[0]*100, df_G1_G2.shape[0]/not_similar_cell_cycle.shape[0]*100)) 
print("Cells with G1/S nuclear cell cycle: %.1f%% (%.1f%%)" %(df_G1_S.shape[0]/df_2_nuclear.shape[0]*100, df_G1_S.shape[0]/not_similar_cell_cycle.shape[0]*100)) 
print("Cells with G2/S nuclear cell cycle: %.1f%% (%.1f%%)" %(df_G2_S.shape[0]/df_2_nuclear.shape[0]*100, df_G2_S.shape[0]/not_similar_cell_cycle.shape[0]*100)) 

Cells with G1/G2 nuclear cell cycle: 3.8% (36.2%)
Cells with G1/S nuclear cell cycle: 2.9% (27.7%)
Cells with G2/S nuclear cell cycle: 3.8% (36.2%)


In [93]:
df_G1_S

,Total Intensity CSU 561,Mean Intensity CSU 561,Total Intensity CSU 488,Mean Intensity CSU 488,Total Intensity CSU 405,Mean Intensity CSU 405,Area,Parent Object ID (MO),cell cycle
517,3788446.2,11.012.925,8566337,24.902.144,7648615.0,22.234.346,3440,1095,S
518,1575226.0,16.974.418,132490,14.276.939,3266659.0,35.201.067,928,1095,G1
519,1466600.0,14.784.274,137185,13.829.134,3234297.0,32.603.801,992,1101,G1
520,4924347.0,1.586.452,6038967,19.455.435,6443770.5,2.075.957,3104,1101,S
523,4834459.0,4472.21.00,210525,19.475.023,3199989.0,29.602.119,1081,1127,G1
524,2561306.0,14.899.977,3881680,22.581.035,4505869.0,26.212.153,1719,1127,S
533,8684240.0,16.397.734,22834784,43.117.041,9642228.0,18.206.624,5296,1203,S
534,4551189.0,19.753.424,568741,2.468.494,4722258.0,20.495.911,2304,1203,G1
549,3946813.0,17.012.125,2172770,93.653.882,4858958.0,20.943.784,2320,1551,S
550,2276592.0,10.462.279,247174,113.591,4606400.0,21.169.119,2176,1551,G1
